In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
import math
import copy
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.metrics import precision_score, recall_score

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [3]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [8]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
# Set your custom dataset path
data_dir = "gdrive/MyDrive/Colab Notebooks/ucb_capstone_asd_detection/data_transformer/"
input_file = 'input.pt'
label_file = 'labels.pt'

# Create the full file path
input_path = f"{data_dir}/{input_file}"
label_path = f"{data_dir}/{label_file}"

# Read the CSV file into a DataFrame
input = torch.load(input_path)
label = torch.load(label_path)

In [10]:
label = torch.where(label != 0, torch.tensor(1), label)

In [11]:
# Combine input and label into a TensorDataset
dataset = TensorDataset(input, label)

# Specify the percentage of data to be used for validation (e.g., 20%)
validation_percentage = 0.2
val_size = int(len(dataset) * validation_percentage)
train_size = len(dataset) - val_size

# Randomly split the dataset into training and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=25, shuffle=False)

In [12]:
input_vocab_size = 5000
label_vocab_size = 2
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(input_vocab_size, label_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)


In [13]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [14]:
def calculate_accuracy(predictions, targets):
    """
    Calculate accuracy given the predicted and target labels.

    Args:
    - predictions (torch.Tensor): Predicted labels
    - targets (torch.Tensor): True labels

    Returns:
    - accuracy (float): Accuracy value between 0 and 1
    """
    with torch.no_grad():
        #_, predicted = torch.max(predictions, 1)
        predicted = torch.where(torch.argmax(predictions.squeeze(dim=1), dim=1) == 0, 1, 0)
        #predicted = torch.argmax(predictions, dim=1)
        correct = (predicted == targets).sum().item()
        total = targets.size(0)
        accuracy = correct / total
    return accuracy

In [15]:
for epoch in range(5):

  # Training
  transformer.train()
  for batch_idx, (train_input, train_label) in enumerate(train_dataloader):
      optimizer.zero_grad()
      output = transformer(train_input, train_label)
      loss = criterion(output.contiguous().view(-1, label_vocab_size), train_label.contiguous().view(-1))
      #loss = criterion((torch.where(torch.argmax(output.squeeze(dim=1), dim=1) == 0, 1, 0)).to(dtype=torch.float), train_label.contiguous().view(-1))
      loss.backward()
      optimizer.step()

  # Validation
  transformer.eval()
  val_loss = 0.0
  val_accuracy = 0

  with torch.no_grad():
    for batch_idx, (val_input, val_label) in enumerate(val_dataloader):
      val_output = transformer(val_input, val_label)
      val_loss += criterion(val_output.contiguous().view(-1, label_vocab_size), val_label.contiguous().view(-1))
      #val_loss += criterion(torch.where(torch.argmax(val_output.squeeze(dim=1), dim=1) == 0, 1, 0), val_label.contiguous().view(-1))

      # Calculate accuracy (you need to define an appropriate accuracy function)
      val_accuracy += calculate_accuracy(val_output, val_label)

    val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_accuracy / len(val_dataloader)

    # Convert tensors to NumPy arrays
    #y_true_np = val_label.numpy()
    #y_pred_np = torch.where(torch.argmax(val_output.squeeze(dim=1), dim=1) == 0, 1, 0).numpy()

      # Calculate precision and recall
     # precision = precision_score(y_true_np, y_pred_np)
      #recall = recall_score(y_true_np, y_pred_np)

  print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")
  # Print the results
  #print(f'Precision: {precision:.4f}')
  #print(f'Recall: {recall:.4f}')
  #print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

Epoch: 1, Training Loss: 3.484578883217182e-07, Validation Loss: 2.3841855067985307e-07, Validation Accuracy: 12.4
Epoch: 2, Training Loss: 1.1920928244535389e-07, Validation Loss: 1.1920927533992653e-07, Validation Accuracy: 12.4
Epoch: 3, Training Loss: 1.1920927533992653e-07, Validation Loss: 1.1920927533992653e-07, Validation Accuracy: 12.4
Epoch: 4, Training Loss: 6.357828397085541e-08, Validation Loss: 0.0, Validation Accuracy: 12.4
Epoch: 5, Training Loss: 0.0, Validation Loss: 0.0, Validation Accuracy: 12.4
